In [32]:
import numpy as np
import pandas as pd
# Dados séries temporai
from datetime import datetime, timedelta
import pytz
# Integração com MetaTrader 5
import datetime as dt
import MetaTrader5 as mt5
import warnings
warnings.filterwarnings("ignore")


In [33]:
mt5.initialize()

True

In [34]:
def baixar_dados(ticker):
    cotacoes_mt5_all = []

    timezone = pytz.timezone("America/Sao_Paulo")
    data_inicio = datetime.now() - timedelta(days=1800)
    data_fim = datetime.now()

    cotacoes_mt5_ticker = mt5.copy_rates_range(ticker, mt5.TIMEFRAME_D1, data_inicio, data_fim)
    cotacoes_mt5_ticker = pd.DataFrame(cotacoes_mt5_ticker)
    cotacoes_mt5_ticker['ticker'] = str(ticker)
    cotacoes_mt5_ticker.index = pd.to_datetime(cotacoes_mt5_ticker['time'], unit='s')
    cotacoes_mt5_all.append(cotacoes_mt5_ticker)
    return pd.concat(cotacoes_mt5_all, axis=0)

In [35]:
def converter_timestamp_em_datetime(timestamp):
        return datetime.utcfromtimestamp(timestamp)

def create_perc(df_ativo, percentual):

    #Ajustar variáveis
    df_ativo['fechAnt'] = df_ativo['close'].shift(1)
    df_ativo['entrada'] = df_ativo['fechAnt'] - (df_ativo['fechAnt'] * percentual)
    df_ativo['percentual'] = percentual

    

    # Aplicar a função de conversão à coluna 'time' do DataFrame
    df_ativo['data'] = df_ativo['time'].apply(converter_timestamp_em_datetime)


    return df_ativo

In [36]:
def create_stats (df_ativo, dias):
    df_ativo['tem_entrada'] = np.where(df_ativo['low'] < df_ativo['entrada'],1,0)

    df_ativo['alvo'] = np.where(df_ativo['tem_entrada'] == 1 , (df_ativo['close'] / df_ativo['entrada'] -1) *100, 0) 
    df_ativo['alvo'] =  np.where(df_ativo['open'] < df_ativo['entrada'], (df_ativo['close'] / df_ativo['open'] - 1) * 100, df_ativo['alvo'] )

    df_filtrado = df_ativo.loc[df_ativo['tem_entrada']==1]
    df_filtrado = df_filtrado.tail(dias)
    df_filtrado['alvo_acumulado'] = df_filtrado['alvo'].cumsum()
    return df_filtrado

In [37]:
def calcular_drawdown(df_filtrado):
    df_filtrado['max_acumulado'] = df_filtrado['alvo_acumulado'].cummax()
    df_filtrado['drawdown'] = df_filtrado['alvo_acumulado'] - df_filtrado['max_acumulado']

    # Calcular o drawdown máximo
    drawdown_maximo = df_filtrado['drawdown'].min()

    # Localizar o índice do valor mínimo em 'drawdown'
    indice_drawdown_maximo = df_filtrado['drawdown'].idxmin()


    # Calcular a taxa de acerto
    resultado = df_filtrado['alvo'] > 0
    taxa_acerto = sum(resultado) / len(df_filtrado['alvo'])

    # Separar resultados positivos e negativos
    resultados_positivos = df_filtrado.loc[df_filtrado['alvo'] > 0, 'alvo']
    resultados_negativos = df_filtrado.loc[df_filtrado['alvo'] <= 0, 'alvo']

    try:
        pior_dia = min(resultados_negativos)
        melhor_dia = max(resultados_positivos)
    except:
        pior_dia = 0
        melhor_dia = 0

    pior_dia = min(resultados_negativos)
    
    quantidade_trades = len(resultados_positivos) + len(resultados_negativos)
    # Calcular médias
    media_positivos = resultados_positivos.mean()
    media_negativos = resultados_negativos.mean()

    # Calcular frequência de resultados positivos e negativos
    freq_positivos = len(resultados_positivos) / len(df_filtrado)
    freq_negativos = len(resultados_negativos) / len(df_filtrado)

    # Calcular expectativa matemática
    expectativa = freq_positivos * media_positivos - freq_negativos * abs(media_negativos)

    # Calcular retorno médio diário
    retorno_medio_diario = df_filtrado['alvo'].mean()

    # Calcular desvio padrão diário
    desvio_padrao_diario = df_filtrado['alvo'].std()

    # Calcular índice de Sharpe
    indice_sharpe = np.sqrt(252) * retorno_medio_diario / desvio_padrao_diario
    
    ativo =  df_filtrado.iloc[2]['ticker']
    retorno_acumulado = df_filtrado.iloc[-1]['alvo_acumulado']


    indice_pior_dia = df_filtrado['alvo'].idxmin()
    media_geral = df_filtrado['alvo'].mean()

    data_pior_dia = df_filtrado.loc[indice_pior_dia, 'data']
    primeiro_dia = df_filtrado.iloc[1]['data']

    resultado_dict = {
        'ativo': ativo,
        'percentual': df_filtrado.iloc[2]['percentual'] * 100,
        'Retorno Acumulado': retorno_acumulado,
        'Melhor dia': melhor_dia,
        'Pior Dia':pior_dia, 
        'Data Pior dia': data_pior_dia,
        'Pior Dia': pior_dia,
        'Primeiro Dia': primeiro_dia,
        'Quantidade de Trades': quantidade_trades,
        'Taxa de Acerto': taxa_acerto,
        'Media Geral': media_geral,
        'Média Positivos': media_positivos,
        'Média Negativos': media_negativos,
        'Expectativa Matemática': expectativa,
        'Índice de Sharpe': indice_sharpe,
        'Drawdown Máximo': drawdown_maximo
    }

    return resultado_dict


In [38]:
ibov = ['CEDO4',
'SGPS3',
'FHER3',
'OSXB3',
'MEAL3',
'SHOW3',
'CEAB3',
'MDNE3',
'AALR3',
'TCSA3',
'PDTC3',
'DMVF3',
'HBSA3',
'GMAT3',
'SMFT3',
'HBRE3',
'MBLY3',
'WEST3',
'OPCT3',
'ELMD3',
'ANIM3',
'ENEV3',
'EMBR3',
'TEND3',
'NINJ3',
'CVCB3',
'CLSA3',
'ONCO3',
'KRSA3',
'ALPK3',
'SEER3',
'LJQQ3',
'HAPV3',
'ALPA4',
'ALPA3',
'LOGN3',
'AERI3',
'GOLL4',
'BRKM5',
'BRKM3',
'NTCO3',
'PCAR3',
'YDUQ3',
'TRAD3',
'SCAR3',
'AMBP3',
'QUAL3',
'CSAN3',
'MLAS3',
'SYNE3',
'MGLU3',
'COGN3',
'JALL3',
'IFCM3',
'ESPA3',
'MRVE3',
'PRIO3',
'GUAR3',
'MEGA3',
'ZAMP3',
'AGRO3',
'PSSA3',
'JBSS3',
'EVEN3',
'CPLE6',
'ECOR3',
'UCAS3',
'AESB3',
'PRNR3',
'SMTO3',
'DIRR3',
'ARZZ3',
'SUZB3',
'RDOR3',
'FLRY3',
'FESA4',
'MRFG3',
'PFRM3',
'DXCO3',
'FRAS3',
'VBBR3',
'RAPT4',
'RAPT3',
'BRFS3',
'TRPL3',
'TRPL4',
'DASA3',
'PNVL3',
'VVEO3',
'BPAN4',
'JSLG3',
'ALLD3',
'TOTS3',
'SAPR11',
'ABCB4',
'ORVR3',
'EUCA4',
'EUCA3',
'BPAC11',
'VLID3',
'IRBR3',
'TECN3',
'PGMN3',
'PINE4',
'LOGG3',
'TTEN3',
'BBSE3',
'CAML3',
'UGPA3',
'PDGR3',
'POMO4',
'POMO3',
'FIQE3',
'BMEB3',
'BMEB4',
'BBAS3',
'VALE3',
'GOAU3',
'LAVV3',
'GOAU4',
'RAIZ4',
'WEGE3',
'MDIA3',
'LAND3',
'GFSA3',
'GGBR3',
'GGBR4',
'LREN3',
'ENGI11',
'CSMG3',
'EZTC3',
'BMOB3',
'BRBI11',
'ITSA3',
'ITSA4',
'MYPK3',
'MTRE3',
'B3SA3',
'RENT3',
'INTB3',
'HYPE3',
'MULT3',
'LUPA3',
'VULC3',
'ARML3',
'ROMI3',
'GGPS3',
'AVLL3',
'TGMA3',
'CRFB3',
'OFSA3',
'ALUP11',
'BEEF3',
'RAIL3',
'BLAU3',
'CCRO3',
'RRRP3',
'LVTC3',
'JHSF3',
'ODPV3',
'IGTI11',
'BRIT3',
'PETZ3',
'CASH3',
'BMGB4',
'RADL3',
'RDNI3',
'TIMS3',
'RECV3',
'TRIS3',
'CBAV3',
'BRAP4',
'ASAI3',
'BRAP3',
'VIVA3',
'TFCO4',
'CMIG4',
'LEVE3',
'PETR4',
'PETR3',
'PORT3',
'USIM5',
'NEOE3',
'MILS3',
'SOJA3',
'CMIG3',
'ELET3',
'ELET6',
'SEQL3',
'CXSE3',
'USIM3',
'NGRD3',
'CSED3',
'EQTL3',
'TASA3',
'APER3',
'TASA4',
'DEXP3',
'DEXP4',
'LPSB3',
'WIZC3',
'MATD3',
'MELK3',
'POSI3',
'AGXY3',
'DESK3',
'HBOR3',
'TUPY3',
'SBSP3',
'SOMA3',
'CPFE3',
'CURY3',
'SBFG3',
'PLPL3',
'ENAT3',
'CSUD3',
'ITUB3',
'ITUB4',
'CLSC4',
'CYRE3',
'GRND3',
'BBDC3',
'BBDC4',
'BRSR6',
'MOVI3',
'STBP3',
'TAEE11',
'PTBL3',
'BRSR3',
'SIMH3',
'AZUL4',
'AURE3',
'CIEL3',
'VITT3',
'VAMO3',
'KLBN11',
'SLCE3',
'RANI3',
'EGIE3',
'CMIN3',
'LWSA3',
'KEPL3',
'ENJU3',
'DOTZ3',
'TPIS3',
'VSTE3',
'VIVR3',
'BHIA3',
'ALOS3'
]

In [39]:
lista_perc = [0.01, 0.015, 0.02, 0.025, 0.03, 0.035, 0.04]
lista_resultados = []
for ativo in ibov:
    print( f'Baixando os dados de {ativo}')
    for perc in lista_perc:
        try:
            df_ativo = baixar_dados(ativo)
            df_ativo = create_perc(df_ativo, perc)
            df_ativo = create_stats(df_ativo, 30)
            resultado = calcular_drawdown(df_ativo)
            lista_resultados.append(resultado)
        except:
            continue

df_resultado = pd.DataFrame(lista_resultados)

Baixando os dados de CEDO4
Baixando os dados de SGPS3
Baixando os dados de FHER3
Baixando os dados de OSXB3
Baixando os dados de MEAL3


Baixando os dados de SHOW3
Baixando os dados de CEAB3
Baixando os dados de MDNE3
Baixando os dados de AALR3
Baixando os dados de TCSA3
Baixando os dados de PDTC3
Baixando os dados de DMVF3
Baixando os dados de HBSA3
Baixando os dados de GMAT3
Baixando os dados de SMFT3
Baixando os dados de HBRE3
Baixando os dados de MBLY3
Baixando os dados de WEST3
Baixando os dados de OPCT3
Baixando os dados de ELMD3
Baixando os dados de ANIM3
Baixando os dados de ENEV3
Baixando os dados de EMBR3
Baixando os dados de TEND3
Baixando os dados de NINJ3
Baixando os dados de CVCB3
Baixando os dados de CLSA3
Baixando os dados de ONCO3
Baixando os dados de KRSA3
Baixando os dados de ALPK3
Baixando os dados de SEER3
Baixando os dados de LJQQ3
Baixando os dados de HAPV3
Baixando os dados de ALPA4
Baixando os dados de ALPA3
Baixando os dados de LOGN3
Baixando os dados de AERI3
Baixando os dados de GOLL4
Baixando os dados de BRKM5
Baixando os dados de BRKM3
Baixando os dados de NTCO3
Baixando os dados de PCAR3
B

In [40]:
df_resultado = pd.DataFrame(lista_resultados)

In [41]:
selecionados = df_resultado.loc[(df_resultado['Taxa de Acerto'] >= 0.75) 
                    & (df_resultado['Pior Dia'] > -1.5) 
                    ].sort_values(['Retorno Acumulado'], ascending=False)
selecionados

,ativo,percentual,Retorno Acumulado,Melhor dia,Pior Dia,Data Pior dia,Primeiro Dia,Quantidade de Trades,Taxa de Acerto,Media Geral,Média Positivos,Média Negativos,Expectativa Matemática,Índice de Sharpe,Drawdown Máximo
1203,BRAP4,4.0,33.556371,4.951923,-1.471084,2022-07-14,2021-12-17,30,0.866667,1.118546,1.395889,-0.684183,1.118546,10.864251,-1.471084
1357,EQTL3,4.0,32.051835,5.392157,-1.305742,2020-05-07,2020-05-07,30,0.833333,1.068394,1.453908,-0.859171,1.068394,10.472628,-1.453973
1174,TIMS3,3.5,32.026387,8.896110,-0.577491,2023-03-31,2021-02-18,30,0.800000,1.067546,1.432991,-0.394233,1.067546,9.288486,-0.698540
690,BPAC11,3.0,27.883114,3.520111,-0.993310,2023-07-06,2023-03-01,30,0.766667,0.929437,1.356748,-0.474585,0.929437,11.567482,-0.993310
577,RAPT3,2.5,24.186006,3.639198,-1.458019,2023-08-03,2022-11-25,30,0.800000,0.806200,1.173265,-0.662058,0.806200,10.797742,-1.458019
1619,AURE3,2.0,18.542246,2.507466,-1.207542,2023-05-08,2022-12-14,30,0.800000,0.618075,0.949106,-0.706049,0.618075,10.219026,-1.207542


### FILTROS EM AÇÕES

In [42]:
# ativo = 'TIMS3'
# lista_perc = [0.035, 0.04]
# lista_resultados = []
# for perc in lista_perc:
#     df_ativo = baixar_dados(ativo)
#     df_ativo = create_perc(df_ativo, perc)
#     df_ativo = create_stats(df_ativo, 30)
#     resultado = calcular_drawdown(df_ativo)
#     lista_resultados.append(resultado)

# df_resultado = pd.DataFrame(lista_resultados)
# df_resultado